# EEG Recordings in pediatric patients with an epilepsy diagnosis based on the 10-20 system

In [5]:
import mne
import numpy as np
import pandas as pd
from scipy.signal import coherence, hilbert
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import scipy.io
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.cm as cm
from matplotlib import colormaps
from networkx.algorithms.community import greedy_modularity_communities, modularity
import os

In [14]:
# creamos una función para leer los archivos
def leer_archivos_de_folder(carpeta_path):
    eeg_files_path = []
    for root, dirs, files in os.walk(carpeta_path):
        for file in files:
            if file.endswith(".edf"):
                eeg_files_path.append(os.path.join(root, file))
    return eeg_files_path

In [16]:
leer_archivos_de_folder(r'C:\Users\jumma\OneDrive\Documentos\GitHub\Neurociencias-2026-1\S03_datasets\proyecto final\task-hfo-eeg')

['C:\\Users\\jumma\\OneDrive\\Documentos\\GitHub\\Neurociencias-2026-1\\S03_datasets\\proyecto final\\task-hfo-eeg\\sub-01_ses-01_task-hfo_eeg.edf',
 'C:\\Users\\jumma\\OneDrive\\Documentos\\GitHub\\Neurociencias-2026-1\\S03_datasets\\proyecto final\\task-hfo-eeg\\sub-02_ses-01_task-hfo_eeg.edf',
 'C:\\Users\\jumma\\OneDrive\\Documentos\\GitHub\\Neurociencias-2026-1\\S03_datasets\\proyecto final\\task-hfo-eeg\\sub-03_ses-01_task-hfo_eeg.edf',
 'C:\\Users\\jumma\\OneDrive\\Documentos\\GitHub\\Neurociencias-2026-1\\S03_datasets\\proyecto final\\task-hfo-eeg\\sub-04_ses-01_task-hfo_eeg.edf',
 'C:\\Users\\jumma\\OneDrive\\Documentos\\GitHub\\Neurociencias-2026-1\\S03_datasets\\proyecto final\\task-hfo-eeg\\sub-05_ses-01_task-hfo_eeg.edf',
 'C:\\Users\\jumma\\OneDrive\\Documentos\\GitHub\\Neurociencias-2026-1\\S03_datasets\\proyecto final\\task-hfo-eeg\\sub-06_ses-01_task-hfo_eeg.edf',
 'C:\\Users\\jumma\\OneDrive\\Documentos\\GitHub\\Neurociencias-2026-1\\S03_datasets\\proyecto final\\task

In [17]:
# creamos una función para procesar archivos
def cargayprocesamiento(eeg_file_path, fmin=8, fmax=13):
    raw = mne.io.read_raw_edf(eeg_file_path, preload=True)
    sfreq_target = raw.info['sfreq']
    raw.resample(sfreq_target)
    raw.filter(1., 45., fir_design='firwin')
    eeg_data = raw.get_data()
    return eeg_data

In [10]:
# creamos una función para convertir a diccionario
def convertir_a_dict(folder_path):
    for path in eeg_files_path:
        
    